# Исследования по классификации текстовых данных

## Этап 1 - Подготовка 

In [1]:
import os
from pathlib import Path

DATA_DIR = Path("data")
if not DATA_DIR.exists():
    raise Exception("Data directory '%s' not found" % DATA_DIR.absolute())
    
OUT_DIR = Path("out_classification")
if not OUT_DIR.exists():
    OUT_DIR.mkdir()    

In [2]:
import numpy as np
import pandas as pd

## Этап 2 - Загрузка данных

In [3]:
complaints_file = os.path.join(DATA_DIR, "complaints.xlsx")

raw_data = pd.read_excel(complaints_file, sheet_name="complaints", index_col=None)

In [4]:
raw_data.head(3)

,ISSUENUM,TOPIC,DESCRIPTION,UNDER_TOPIC,REPORTER,ASSIGNEE,CREATOR,STATUS_ISSUE,CREATED,UPDATED,...,FIO_CLIENT,IIN,TEXT_CONTRACT_NUMBER,PHONE_NUMBER,INF_CLIENT_RESPONDING,INCIDENT_REGION,INCIDENT_RESOLVED,FIO_GUILT_WORK,AUTHOR_REGION,AUTHOR_SUBDIVISIONS
0,105578,ЖАЛОБА: на корреспонденцию банка,Клиент просит чтобы банк не отправлял смс о сп...,Cмс по предложениям Xsell,aoryntaeva,akalabaeva,aoryntaeva,Закрыта,2017-12-12 06:47:45,2017-12-12 12:02:50,...,АҚЖИГИТОВ БЕЙБІТ КЕНЖЕБАЙҰЛЫ,910429301833,0,7776121291,NaN,0,Не требует создания проблемы,NaN,Алматы,Call-center
1,105582,ЖАЛОБА: на корреспонденцию банка,ДД.\nПри оформлении кредита клиент предоставил...,Cмс по предложениям Xsell,atusipova,akalabaeva,atusipova,Закрыта,2017-12-12 08:34:06,2017-12-12 15:41:16,...,Тверитинова Анна Егоровна,761123400947,3708168706,87078452136 \t,NaN,0,Не требует создания проблемы,NaN,NaN,Call-center
2,105604,ЖАЛОБА: на обслуживание,Дарипова Айгуль Сабыргалиевна жалуется на мене...,NaN,azhalgaeva,zissayev,azhalgaeva,Закрыта,2017-12-12 09:27:28,2017-12-21 09:10:29,...,Дарипова Айгуль Сабыргалиевна,810420403003,0,7292250739,Не дозвон,"Мангистауская область,Бейнеу, ул. Ерконай, д. 1",Не требует создания проблемы,NaN,Актау,Call-center


In [5]:
print("[i] Размерность сырых данных: ", raw_data.shape)

[i] Размерность сырых данных:  (22840, 21)


## Этап 3 - Отбор признаков

In [6]:
valuable_columns = ["ISSUENUM", "TOPIC", "DESCRIPTION"]
raw_data = raw_data[valuable_columns] # Для исследования ограничимся ID 'ISSUENUM', меткой 'TOPIC' и содержанием 'DESCRIPTION'

print("[i] : Информация о данных")
raw_data.info() # Информация о данных

[i] : Информация о данных
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22840 entries, 0 to 22839
Data columns (total 3 columns):
ISSUENUM       22840 non-null int64
TOPIC          22805 non-null object
DESCRIPTION    22839 non-null object
dtypes: int64(1), object(2)
memory usage: 535.4+ KB


In [7]:
raw_data.head(3)

,ISSUENUM,TOPIC,DESCRIPTION
0,105578,ЖАЛОБА: на корреспонденцию банка,Клиент просит чтобы банк не отправлял смс о сп...
1,105582,ЖАЛОБА: на корреспонденцию банка,ДД.\nПри оформлении кредита клиент предоставил...
2,105604,ЖАЛОБА: на обслуживание,Дарипова Айгуль Сабыргалиевна жалуется на мене...


## Этап 4 - Проверка на дубли и удаление строк с отсутствующими данными

In [8]:
duplicated = raw_data.loc[raw_data.duplicated(subset=['ISSUENUM'], keep=False)]
print("[i] Количество дублей идентификаторов: ", duplicated['ISSUENUM'].count())

[i] Количество дублей идентификаторов:  0


In [9]:
raw_data = raw_data.dropna(subset=['TOPIC', 'DESCRIPTION']) # Удаляем строки с пустыми значениями в столбцах 'TOPIC' и 'DESCRIPTION'

print("[i] Размерность после удаления пустых", raw_data.shape)

[i] Размерность после удаления пустых (22804, 3)


## Этап 5 - Отбор "нужных" обращений

In [10]:
#Отберем только строки, содержащие жалобы, в которых столбец 'TOPIC' содержит различные сочетания слова "жалоба"
raw_data = raw_data[(raw_data.TOPIC.str.contains("ЖАЛОБА: ") |
                     raw_data.TOPIC.str.contains("Жалоба на ") |
                     raw_data.TOPIC.str.contains("Жалобы на "))] #, na=False

print("[i] Размерность данных с Жалобами", raw_data.shape)

[i] Размерность данных с Жалобами (19415, 3)


## Этап 6 - Очистка меток от "мусора"

In [11]:
import re
def label_cleaner(text):
        
    text = re.sub( r'ЖАЛОБА: на ', '', text) # Убрать из темы словосочетания 'ЖАЛОБА: на ',
    text = re.sub( r'ЖАЛОБА: ', '', text) # 'ЖАЛОБА: ',
    text = re.sub( r'Жалоба на ', '', text) # 'Жалоба на ',
    text = re.sub( r'Жалобы на ', '', text) # 'Жалобы на '
    
    text = str.strip(text.lower()) #Переводим в нижний регистр и убираем пробелы в начале и конце
    return  text

In [12]:
raw_data["TOPIC"] = [label_cleaner(t) for t in raw_data["TOPIC"]] # Почистим метки от "мусора"

print("[i] Количество категорий (меток):", len(set(raw_data["TOPIC"])))
print("[i] Множество категорий (меток):")
print(set(raw_data["TOPIC"]))

[i] Количество категорий (меток): 9
[i] Множество категорий (меток):
{'услугу "хранитель"', 'услугу "защита семьи"', 'обслуживание', 'карточные продукты', 'не согласие с условиями договора, %%, задолженностью, штрафами, тарифами и комиссиями', 'го/филиал/отделения/микроофисы/тт', 'обслуживание в терминалах', 'корреспонденцию банка', 'услугу "страховку"'}


## Этап 7 - Препроцессинг

In [13]:
#Пишем свой препроцессор (preprocessor)
from nltk.stem.snowball import SnowballStemmer
def text_cleaner(text):
    text = text.lower() # приведение в lowercase,
    
    stemmer = SnowballStemmer("russian")
    singles = [stemmer.stem(word) for word in text.split()]
    text = ' '.join(singles)

    #Удаление незначимых слов
    stw = ['в', 'по', 'на', 'из', 'и', 'или', 'не', 'но', 'за', 'над', 'под', 'то',
           'a', 'at', 'on', 'of', 'and', 'or', 'in', 'for', 'at' ]
    remove = r'\b(' + '|'.join(stw) + r')\b'  
    text = re.sub(remove, ' ', text)
    
    text = re.sub( r'\b\w\b', '', text ) # удаление отдельно стоящих букв

    text = re.sub( r'\b\d+\b', ' digit ', text ) # замена цифр 

    return  text

#raw_data["DESCRIPTION"] = [text_cleaner(t) for t in raw_data["DESCRIPTION"]]
#raw_data["DESCRIPTION"].head(5)

## Этап 8 - Разделение на тренировочную и тестовую выборки

In [14]:
def train_test_split( data, validation_split = 0.2):
    dict_data = data.to_dict('list')
    sz = len(dict_data['TOPIC'])
    indices = np.arange(sz)
    np.random.shuffle(indices)

    I = [dict_data['ISSUENUM'][i] for i in indices ]
    X = [dict_data['DESCRIPTION'][i] for i in indices]
    Y = [dict_data['TOPIC'][i] for i in indices]
    
    nb_validation_samples = int(validation_split * sz)

    return { 
        'train': { 'i': I[:-nb_validation_samples], 'x': X[:-nb_validation_samples], 'y': Y[:-nb_validation_samples]  },
        'test': { 'i': I[-nb_validation_samples:], 'x': X[-nb_validation_samples:], 'y': Y[-nb_validation_samples:]  }
    }

In [15]:
#Разделяем данные на Тренировочную и Тестовую выборки
D = train_test_split(raw_data, 0.3)

print("[i] Размер тестовой выборки:", len(D['test'] ['y']))
print("[i] Размер тренировочной выборки:", len(D['train'] ['y']))
print("[i] Общий размер данных:", len(D['test'] ['y']) + len(D['train'] ['y']))

[i] Размер тестовой выборки: 5824
[i] Размер тренировочной выборки: 13591
[i] Общий размер данных: 19415


## Этап 9 - Обучение классификатора 

In [16]:
#Проверим версию scikit-learn:
import sklearn
print('[i] Версия scikit-learn: ', sklearn.__version__)

[i] Версия scikit-learn:  0.20.3


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

vectorizer = TfidfVectorizer() # preprocessor=text_cleaner. В данном случае, почему-то, препроцессинг не повышает точность...
print(vectorizer)
print("\n")
classifier = SGDClassifier(loss='hinge', tol=0.0001) #, max_iter=20, tol=0.0001, verbose = 1
print(classifier)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=0.0001,
       validation_fraction=0.1, verbose=0, warm_start=False)


In [18]:
print("[i] Обучение классификатора...")

text_clf = Pipeline([
                ('tfidf', vectorizer),
                ('clf', classifier),
                ])

text_clf.fit(D['train']['x'], D['train']['y'])

print("[i] Обучение завершено!")

[i] Обучение классификатора...
[i] Обучение завершено!


## Этап 10 - Тестирование

In [19]:
from sklearn.metrics import accuracy_score
#Testing and checking results
print("[i] Тестируем...")

predicted_train = text_clf.predict( D['train']['x'] )
print("[i] accuracy train: ", accuracy_score(  D['train']['y'] , predicted_train) )
    
predicted_test = text_clf.predict( D['test']['x'] )
print("[i] accuracy test: ", accuracy_score(  D['test']['y'] , predicted_test) )

[i] Тестируем...
[i] accuracy train:  0.9173717901552498
[i] accuracy test:  0.859375


## Этап 11 - Сохранение результатов

In [20]:
frameTest = pd.DataFrame({'id' : D['test']['i'],'cathegory': D['test']['y'], 'content': D['test']['x']})
frameTest['predicted'] = predicted_test.tolist()

In [21]:
frameTest.head(10)

,id,cathegory,content,predicted
0,201638,обслуживание,"Клиенту в торговой точке не объяснили, что пер...",обслуживание
1,164892,обслуживание,клиент жалуется что оператор предоставила не к...,обслуживание
2,149685,корреспонденцию банка,Клиент УВАЛИЕВА ГУЛЬНАРА АСЫЛБЕКОВНА жалуется ...,корреспонденцию банка
3,147971,обслуживание,Добрый день!\nУ клиента кредитный договор офор...,обслуживание
4,116855,корреспонденцию банка,Нейфельд Галина\nАнатолиевна приходит смс сооб...,корреспонденцию банка
5,131283,корреспонденцию банка,ДД! Позвонил 3 лицо сказал что звонили с нашег...,корреспонденцию банка
6,107603,обслуживание,ЗК возмущается в связи с тем что она не знала ...,обслуживание
7,125982,корреспонденцию банка,Клиент Рысмагамбетов Назар Акбарович просит не...,корреспонденцию банка
8,316694,"услугу ""страховку""",Клиент утверждает что кредитный консультант не...,"услугу ""страховку"""
9,224005,го/филиал/отделения/микроофисы/тт,Клиент жалуется на количество обслуживающего п...,обслуживание


In [22]:
#Сохраняем в файл
frameTest.to_excel(os.path.join(OUT_DIR, "complaints_test.xlsx"), sheet_name='complaints_test')